In [44]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to C:\Users\sandhya yenimi
[nltk_data]     reddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\sandhya yenimi
[nltk_data]     reddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [46]:
data= pd.read_csv('./data1/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

In [47]:
data.head()

essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   rater1_domain1  rater2_domain1  rater3_domain1  domain1_score  \
0               4               4             NaN              8   
1               5               4             NaN              9   
2               4               3             NaN              7   
3               5               5             NaN             10   
4               4               4             NaN              8   

   rater1_domain2  rater2_domain2  domain2_score  ...  rater2_trait3  \
0             NaN             NaN            NaN  ...            NaN   
1             NaN             NaN            NaN  ...            NaN   
2             NaN             NaN            NaN  ...            NaN   
3             NaN             NaN            NaN  ...            NaN   
4             NaN             NaN            NaN  ...            NaN   

   rater2_trait4  rater2_trait5  rater2_trait6  rater3_trait1  rater3_trait2  \
0            NaN            NaN            NaN            NaN            NaN   
1            NaN            NaN            NaN            NaN            NaN   
2            NaN            NaN            NaN            NaN            NaN   
3            NaN            NaN            NaN            NaN            NaN   
4            NaN            NaN            NaN            NaN            NaN   

   rater3_trait3  rater3_trait4  rater3_trait5  rater3_trait6  
0            NaN            NaN            NaN            NaN  
1            NaN            NaN            NaN            NaN  
2            NaN            NaN            NaN            NaN  
3            NaN            NaN            NaN            NaN  
4            NaN            NaN            NaN            NaN  

[5 rows x 28 columns]

In [48]:
data.isnull().any()

essay_id          False
essay_set         False
essay             False
rater1_domain1    False
rater2_domain1    False
rater3_domain1     True
domain1_score     False
rater1_domain2     True
rater2_domain2     True
domain2_score      True
rater1_trait1      True
rater1_trait2      True
rater1_trait3      True
rater1_trait4      True
rater1_trait5      True
rater1_trait6      True
rater2_trait1      True
rater2_trait2      True
rater2_trait3      True
rater2_trait4      True
rater2_trait5      True
rater2_trait6      True
rater3_trait1      True
rater3_trait2      True
rater3_trait3      True
rater3_trait4      True
rater3_trait5      True
rater3_trait6      True
dtype: bool

In [49]:
data= data.dropna(axis=1)
data= data.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [50]:
data.head()

essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  
0              8  
1              9  
2              7  
3             10  
4              8

In [51]:
x=data.iloc[:,0:3]
y=data.iloc[:,3]

In [52]:
x

essay_id  essay_set                                              essay
0             1          1  Dear local newspaper, I think effects computer...
1             2          1  Dear @CAPS1 @CAPS2, I believe that using compu...
2             3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...
3             4          1  Dear Local Newspaper, @CAPS1 I have found that...
4             5          1  Dear @LOCATION1, I know having computers has a...
5             6          1  Dear @LOCATION1, I think that computers have a...
6             7          1  Did you know that more and more people these d...
7             8          1  @PERCENT1 of people agree that computers make ...
8             9          1  Dear reader, @ORGANIZATION1 has had a dramatic...
9            10          1  In the @LOCATION1 we have the technology of a ...
10           11          1  Dear @LOCATION1, @CAPS1 people acknowledge the...
11           12          1  Dear @CAPS1 @CAPS2 I feel that computers do ta...
12           13          1  Dear local newspaper I raed ur argument on the...
13           14          1  My three detaileds for this news paper article...
14           15          1  Dear, In this world today we should have every...
15           16          1  Dear @ORGANIZATION1, The computer blinked to l...
16           17          1  Dear Local Newspaper, I belive that computers ...
17           18          1  Dear Local Newspaper, I must admit that the ex...
18           19          1  I aegre waf the evansmant ov tnachnolage. The ...
19           20          1  Well computers can be a good or a bad thing. I...
20           21          1  Dear @CAPS1 of the @CAPS2 @CAPS3 daily, I am w...
21           22          1  Dear local Newspaper @CAPS1 a take all your co...
22           23          1  Dear local newspaper, @CAPS1 you ever see a ch...
23           24          1  Dear local newspaper, I've heard that not many...
24           25          1  Dear @CAPS1, @CAPS2 off, I beileve that comput...
25           26          1  Do you think that computers are useless? Or do...
26           27          1  Computers a good because you can get infermati...
27           28          1  Dear Newspaper, Computers are high tec and hav...
28           29          1  Dear local newspaper, @CAPS1 people throughout...
29           30          1  Dear Newspaper People, I think that computers ...
...         ...        ...                                                ...
12946     21592          8   We all understand the benefits of laughter. L...
12947     21594          8        It was midsummer, and i could feel the c...
12948     21595          8   Have you ever experienced a time with your fr...
12949     21596          8   I woke up just like any other day happy yet l...
12950     21598          8   Laughter is an important part of my life, eit...
12951     21599          8   I sat at the table, speechless, as they told ...
12952     21601          8   As I remember back, it was @DATE1. It was a h...
12953     21603          8   Those eyes, it was like I was looking out int...
12954     21604          8  Some say that laugh is the common language bet...
12955     21605          8   Laughter is an integral element to many situa...
12956     21606          8  One time I was at my friend @PERSON1's house, ...
12957     21607          8   LAUGHTER @CAPS1 knows that laughter is a heal...
12958     21608          8  One thing that people in the world love to do ...
12959     21609          8   Laughter, to me, is an important aspect of my...
12960     21610          8   People always say that the worst parts of lif...
12961     21611          8   Why is it that people can look back at someth...
12962     21613          8   Before my best friend moved away, we would st...
12963     21615          8                                @ORGANIZATION1  ...
12964     21617          8   Morose and somnolent, I woke up. I woke up to...
12965     21618          8   

In [53]:
y

0         8
1         9
2         7
3        10
4         8
5         8
6        10
7        10
8         9
9         9
10        8
11        8
12        7
13        6
14        6
15       12
16        8
17        8
18        4
19        6
20        8
21        3
22       10
23       11
24        8
25        9
26        4
27        9
28        9
29        8
         ..
12946    40
12947    32
12948    36
12949    31
12950    30
12951    47
12952    40
12953    35
12954    33
12955    36
12956    36
12957    48
12958    40
12959    40
12960    40
12961    42
12962    40
12963    32
12964    36
12965    40
12966    10
12967    33
12968    44
12969    35
12970    30
12971    35
12972    32
12973    40
12974    40
12975    40
Name: domain1_score, Length: 12976, dtype: int64

In [54]:
min_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

# preprocessing

In [55]:
#removing the extra characters other than alphabets and stopwords and tokenizing the words
def essay_to_wordlist(essay_v):
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

#Tokenize the senstences and call essay_to_wordlist() for word tokenization.
def essay_to_sentences(essay_v):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence))
    return sentences


In [56]:
#Feature vector is made from the words list of an essay.
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

#Word vectors are generated for Word2Vec model
def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [57]:
#the dataset is split to training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [58]:
x_train.shape

(10380, 3)

In [59]:
x_test.shape

(2596, 3)

In [60]:
train_essays = x_train['essay']
test_essays = x_test['essay']

In [61]:
sentences = []
# Obtaining all sentences from the training essays.
for essay in train_essays:
    sentences += essay_to_sentences(essay)

In [62]:
sentences

[['essay', 'author', 'talking', 'journey'],
 ['shows',
  'give',
  'ends',
  'almost',
  'way',
  'began',
  'shows',
  'people',
  'stay',
  'attached',
  'past',
  'reasons',
  'believe',
  'author',
  'concludes',
  'story',
  'paragraph'],
 ['shows',
  'people',
  'give',
  'even',
  'though',
  'failed',
  'test',
  'try',
  'later',
  'pass'],
 ['also',
  'story',
  'ends',
  'way',
  'begins',
  'taking',
  'test',
  'getting',
  'ready',
  'good',
  'way',
  'writing'],
 ['one',
  'last',
  'thing',
  'shows',
  'people',
  'become',
  'attached',
  'past',
  'look',
  'one',
  'thing',
  'reminds',
  'month',
  'fall',
  'apart',
  'takes',
  'test',
  'see',
  'look',
  'familiar',
  'flowers',
  'without',
  'falling',
  'apart'],
 ['story',
  'winter',
  'hibiscus',
  'minfong',
  'ho',
  'think',
  'ending',
  'paragraph',
  'show',
  'saeng',
  'given'],
 ['says', 'come', 'back', 'take', 'test', 'talking', 'hibiscus', 'geese'],
 ['things', 'remind', 'home'],
 ['reminders'

In [63]:
#build the vectorizer with maximum featuroes of 300
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3
model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, 
                 sample = downsampling)             

In [64]:
#save the vectorizer in .bin file
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

In [65]:
#get the training vectors
clean_train_essays = []
for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
 
#get the testing vectors
clean_test_essays = []
for essay_v in test_essays:
    clean_test_essays.append(essay_to_wordlist( essay_v))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
#convert the vectors to numpy array
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

# model building

In [66]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

In [67]:
model = Sequential()
model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
model.add(LSTM(64, recurrent_dropout=0.4))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))

In [68]:
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

In [69]:
model.fit(trainDataVecs, y_train, batch_size=64, epochs=20)

Epoch 1/20
10380/10380 [==============================] - 12s 1ms/step - loss: 54.2663 - mean_absolute_error: 3.6011
Epoch 2/20
10380/10380 [==============================] - 5s 522us/step - loss: 29.6482 - mean_absolute_error: 2.5330
Epoch 3/20
10380/10380 [==============================] - 5s 516us/step - loss: 19.3068 - mean_absolute_error: 2.1423
Epoch 4/20
10380/10380 [==============================] - 5s 521us/step - loss: 13.2128 - mean_absolute_error: 1.8584
Epoch 5/20
10380/10380 [==============================] - 6s 532us/step - loss: 9.9586 - mean_absolute_error: 1.6683
Epoch 6/20
10380/10380 [==============================] - 5s 517us/step - loss: 8.7249 - mean_absolute_error: 1.5975
Epoch 7/20
10380/10380 [==============================] - 5s 528us/step - loss: 8.2273 - mean_absolute_error: 1.5457
Epoch 8/20
10380/10380 [==============================] - 5s 526us/step - loss: 7.7871 - mean_absolute_error: 1.5053
Epoch 9/20
10380/10380 [==============================] - 5s 

In [70]:
testDataVecs.shape

(2596, 1, 300)

In [71]:
y_pred = model.predict(testDataVecs)

In [72]:
y_pred

array([[13.348173 ],
       [16.934616 ],
       [ 2.9268806],
       ...,
       [ 2.0209796],
       [ 2.6250577],
       [ 2.8144622]], dtype=float32)

In [73]:
model.save('final_lstm.h5')

In [74]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test,y_pred)
accuracy

0.9361932360502562

# model testing

In [75]:
import gensim.models.keyedvectors as word2vec

In [76]:
testsen='''Dear local newspaper, I\'ve heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren\'t, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That\'s amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population\'s hand-eye coordination would increase, and who wouldn\'t want thats. Have you ever heard of @LOCATION1? It is a country in @LOCATION2 that no one really knows much about. By looking it up on the computer, you can become an expert on the country! The computer gives us access to a lot of knowledge, and will make us more of people around the world. This can improve our travel. Instead of going to the same place every vacation, you can go to a place you saw online that looks just as exciting. Also, by learning about other countries and the people in them, we gain respect for them. We @MONTH1 also learn about their traditions, and culture. "The internet gives us access to the world," says @PERSON1, "we can learn so many new things that will make us more of people, countries, and the environment." @CAPS1 is right, we can also be aware of the environment. Since global warming is a problem, we can learn how o preserve foreign places by learning about them on the internet. With a click of a mouse you can virtually have the whose world at your fingertips! In a recent study @PERCENT2 of people said they don\'t have time to meet with old friends. The internet, however has made it quick and easy to talk to people online. Websites like @CAPS2, @CAPS3, and @CAPS4 can keep your social life in tact, and can even reunite yu with old friends. For example, @CAPS2 suggest family. You can keep in touch wih many people trough @CAPS2! Also, you can make new friends. Most teens say they are always willing to meet new people, and the internet is a great way to do so! Sites like @CAPS7, and @CAPS8 can can help you find friends by connecting you in a chat room to a random person, just to talk. I once talked with someone on @CAPS7 for over an hour! We found that we both have @CAPS4 accounts and keep in touch there! This also helps you gain confidence because you @MONTH1 have the courage to say something over the computer that you would\'t say real life. The computer can really amp up your social life. As you can see. computers don\'t just "your barain," they imprdove hand-eye coordination, help you learn about far away places, and improve your social life by allowing you to talk to others. Computers benefit the society more than anymore will ever know!'''

In [77]:
testsen

'Dear local newspaper, I\'ve heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren\'t, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That\'s amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population\'s hand-eye coordination would increase,

In [78]:
model = word2vec.KeyedVectors.load_word2vec_format('word2vecmodel.bin', binary=True)
index2word_set = set(model.index_to_key)

In [79]:
testsen2 = re.sub("[^a-zA-Z]", " ", testsen)
testsen2 = essay_v.lower()
featureVec = np.zeros((300,),dtype="float32")
for word in testsen2:
        if word in index2word_set:
            featureVec = np.add(featureVec,model[word])

In [80]:
featureVec.shape

(300,)

In [81]:
avc=featureVec.reshape(1,1,300)

In [82]:
from keras.models  import load_model
model1 = load_model("final_lstm.h5")

In [83]:
y_pred = model1.predict(avc)

In [84]:
y_pred

array([[7.7951703]], dtype=float32)